# What's New in This Version

1. Using color detection without LED

2. Using average of different edges of the marker to calculate PixelsPerMetric

3. Also we used a manual ppm based on the starting point of the Object of Interest (OOI). Therefore for each run, its value    should be considered.

In [1]:
# import the necessary packages
import numpy as np
import cv2
#print(cv2.__version__)
import cv2.aruco as aruco
from scipy.spatial import distance as dist

from imutils import perspective
from imutils import contours
import imutils

### Color Detection
If the code did not detect the object, uncomment the following code and run only the following block to adjust the needed parameters in order to find the object. Then use this numbers in the first lines of the code:

Define the lower and upper bounds of the green color in HSV color space
green_lower = (29, 86, 6)
green_upper = (64, 255, 255)

<p style="color:green;">
Best range for green colors is:
    <br>
green_lower = (24, 66, 0)
    <br>
green_upper = (101, 255, 255)
</p>

In [2]:
# def onTrack1(val):
#     global hueLow
#     hueLow=val
#     #print('Hue Low',hueLow)
# def onTrack2(val):
#     global hueHigh
#     hueHigh=val
#     #print('Hue High',hueHigh)
# def onTrack3(val):
#     global satLow
#     satLow=val
#     #print('Sat Low',satLow)
# def onTrack4(val):
#     global satHigh
#     satHigh=val
#     #print('Sat High',satHigh)
# def onTrack5(val):
#     global valLow
#     valLow=val
#     #print('Val Low',valLow)
# def onTrack6(val):
#     global valHigh
#     valHigh=val
#     #print('Val High',valHigh)
 
# width=640
# height=360
# cam=cv2.VideoCapture(0,cv2.CAP_DSHOW)
# cam.set(cv2.CAP_PROP_FRAME_WIDTH, width)
# cam.set(cv2.CAP_PROP_FRAME_HEIGHT,height)
# cam.set(cv2.CAP_PROP_FPS, 30)
# cam.set(cv2.CAP_PROP_FOURCC,cv2.VideoWriter_fourcc(*'MJPG'))
 
# cv2.namedWindow('myTracker')
# cv2.moveWindow('myTracker',width,0)
 
# hueLow=0
# hueHigh=179
# satLow=0
# satHigh=255
# valLow=0
# valHigh=255
 
# cv2.createTrackbar('Hue Low','myTracker',0,179,onTrack1)
# cv2.createTrackbar('Hue High','myTracker',179,179,onTrack2)
# cv2.createTrackbar('Sat Low','myTracker',0,255,onTrack3)
# cv2.createTrackbar('Sat High','myTracker',255,255,onTrack4)
# cv2.createTrackbar('Val Low','myTracker',0,255,onTrack5)
# cv2.createTrackbar('Val High','myTracker',255,255,onTrack6)
 
# while True:
#     ignore,  frame = cam.read()
#     frameHSV=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
#     lowerBound=np.array([hueLow,satLow,valLow])
#     upperBound=np.array([hueHigh,satHigh,valHigh])
#     myMask=cv2.inRange(frameHSV,lowerBound,upperBound)
#     #myMask=cv2.bitwise_not(myMask)
#     myObject=cv2.bitwise_and(frame,frame,mask=myMask)
#     myObjectSmall=cv2.resize(myObject,(int(width/2),int(height/2)))
#     cv2.imshow('My Object',myObjectSmall)
#     cv2.moveWindow('My Object',int(width/2),int(height))
#     myMaskSmall=cv2.resize(myMask,(int(width/2),int(height/2)))
#     cv2.imshow('My Mask',myMaskSmall)
#     cv2.moveWindow('My Mask',0,height)
#     cv2.imshow('my WEBcam', frame)
#     cv2.moveWindow('my WEBcam',0,0)
#     if cv2.waitKey(1) & 0xff ==ord('q'):
#         break
# cam.release()

## aruco_finder

In [3]:
def find_aruco_marker(img):
    aruco_dict = aruco.Dictionary_get(aruco.DICT_ARUCO_ORIGINAL)

    # Marker detection parameters
    parameters = aruco.DetectorParameters_create()

    # Lists of ids and the corners belonging to each id
    corners, ids, rejected_points = aruco.detectMarkers(img, aruco_dict, parameters=parameters)

    # Draw what the aruco detector sees
    image = img.copy()
    aruco.drawDetectedMarkers(image, corners)
    cv2.imwrite("./output/Marker.jpg", image)

    # return only the first marker on the list
    return corners[0][0]


def get_calibration_data():
    """
    The distortion coefficient retrieved is for a specific camera; images taken on another camera may produce different results.
    """
    # Read Yaml file and retrieve distortion coefficient and camera matrix
    calibration_file = cv2.FileStorage("./calibration.yaml", cv2.FileStorage_READ)
    file_node = calibration_file.getNode('camera_matrix')
    matrix = np.asarray(file_node.mat())
    file_node = calibration_file.getNode('dist_coeff')
    distortion = np.asarray(file_node.mat())

    return matrix, distortion


def calibrate_image(img, matrix, distortion, new_width=1280, new_height=960):

    # start by scaling all input images
    new_img = cv2.resize(img, (new_width, new_height))

    # Get height/width of image
    h, w = new_img.shape[:2]

    # commented out due to bad warping of objects.
    # new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(matrix, distortion, (w, h), 1, (w, h))
    # undistorted_img = cv2.undistort(new_img, matrix, distortion, None, new_camera_mtx)

    undistorted_img = new_img

    # Crop the image to remove the black border (warped distortion)
    # this cropping is too aggressive for our needs, so it is left commented out.
    # x, y, width, height = roi
    # cropped_img = undistorted_img[y:y+height, x:x+width]

    return undistorted_img

## object_size

In [4]:
# USAGE
# python main.py -i ./input/Straps1.jpg
# python main.py -i ./input/Generated/angle75.png

def midpoint(ptA, ptB):
    return (ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5

def objectsize(image,c, pixelsPerMetric):
# 	print("WE RECIEVED:", marker)
# 	# convert image to grayscale, and blur it slightly
# 	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# 	gray = cv2.GaussianBlur(gray, (7, 7), 0)

# 	# perform edge detection, then perform a dilation + erosion to
# 	# close gaps in between object edges
# 	edged = cv2.Canny(gray, 15, 100)
# 	edged = cv2.dilate(edged, None, iterations=1)
# 	edged = cv2.erode(edged, None, iterations=1)

# 	# cv2.imshow("Edges", edged)
# 	cv2.imwrite("./output/edges.jpg", edged)
# 	cv2.waitKey(0)

# 	# find contours in the edge map
# 	cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
# 							cv2.CHAIN_APPROX_SIMPLE)
# 	cnts = imutils.grab_contours(cnts)

# 	# sort the contours from left-to-right and initialize the
# 	# 'pixels per metric' calibration variable
# 	(cnts, _) = contours.sort_contours(cnts)
	boxes = []

	# loop over the contours individually

#     # if the contour is not sufficiently large, ignore it
#     if cv2.contourArea(c) < 100:
#         continue
    # compute the rotated bounding box of the contour
	box = cv2.minAreaRect(c)
	box = cv2.boxPoints(box)
	box = np.array(box, dtype="int")

		# order the points in the contour such that they appear
		# in top-left, top-right, bottom-right, and bottom-left
		# order, then draw the outline of the rotated bounding
		# box
	box = perspective.order_points(box)
	boxes.append(box)

# 	minDist = 99999.
# 	for box in boxes:  # loop through boxes to find the one closest to the aruco marker, set the PPM
# 		distance = abs(box[2][0]-marker[0])+abs(box[2][1]-marker[1])  # rough difference between marker and current box
# 		print("contour:", box[2])
# 		print("distance:", distance)
# 		if distance < minDist:  # Identifies potential markers:  the correct box will have the smallest distance
# 			print("CLOSEST contour:", box[2])
# 			minDist = distance
# 			pixelsPerMetric = dist.euclidean(box[2], box[1]) / 8  # calculates vertical length of the marker

	outName = 1
	for box in boxes:
		orig = image.copy()
		cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)

	# loop over the original points and draw them
		for (x, y) in box:
			cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)

		# unpack the ordered bounding box, then compute the midpoint
		# between the top-left and top-right coordinates, followed by
		# the midpoint between bottom-left and bottom-right coordinates
		(tl, tr, br, bl) = box
		(tltrX, tltrY) = midpoint(tl, tr)
		(blbrX, blbrY) = midpoint(bl, br)

		# compute the midpoint between the top-left and top-right points,
		# followed by the midpoint between the top-righ and bottom-right
		(tlblX, tlblY) = midpoint(tl, bl)
		(trbrX, trbrY) = midpoint(tr, br)

		# draw the midpoints on the image
		cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
		cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
		cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
		cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

		# draw lines between the midpoints
		cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
				 (255, 0, 255), 2)
		cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
				 (255, 0, 255), 2)

		# compute the Euclidean distance between the midpoints
		dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
		dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))


		# compute the size of the object
		dimA = dA / pixelsPerMetric
		dimB = dB / pixelsPerMetric

		# draw the object sizes on the image
		cv2.putText(orig, "{:.1f}cm".format(dimB),
					(int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,
					0.65, (255, 64, 64), 2)
		cv2.putText(orig, "{:.1f}cm".format(dimA),
					(int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
					0.65, (255, 64, 64), 2)
# 		print("Object " + str(outName)+": " + "{:.1f}cm".format(dimB) + " by {:.1f}cm".format(dimA))
		# show the output image
		# cv2.imshow("Image", orig)
		cv2.imwrite("./output/object "+str(outName)+".jpg", orig)
		outName = outName+1
		cv2.waitKey(10)

## transform

In [5]:
def removeEmpty(img):
	# this function removes pitch black space created when the transformed image
	# generates empty space not part of the original.
	h, w, _ = img.shape
	print(h, w)
	cv2.imwrite("./output/no crop.jpg", img)
	gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	cv2.imwrite("./output/no crop bw.jpg", gray)
	minx = 9999999
	maxx = 0
	miny = 9999999
	maxy = 0

	for y in range(h-1):
		for x in range(w-1):
			if gray[y, x] != 0:
				if miny == 9999999:
					miny = y
				if maxx < x:
					maxx = x
				if minx > x:
					minx = x
				maxy = y

	print(miny, maxy, minx, maxx)

	img2 = img[miny+10:maxy-10, minx+10:maxx-10]
	return img2


def GetPadding(img, pts):
	#print("noob",img.shape)
	height, width, _ = img.shape
	return height, height, width, width
	# 310, 3400, 640, 700


def four_point_transform(image, pts):
	# Calculate the space needed on all sides of the aruco marker
	padUp, padDown, padLeft, padRight = GetPadding(image, pts)
	rect = pts
	(tl, tr, br, bl) = rect
	#print(rect)

	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))

	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))

	maxSize = max(int(maxHeight), int(maxWidth))
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[padLeft, padUp],
		[padLeft+maxSize, padUp],
		[padLeft+maxSize, padUp+maxSize],
		[padLeft, padUp+maxSize]], dtype="float32")

	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	#print(M)
	# Apply perspective transform to the image
	warped = cv2.warpPerspective(image, M, (maxSize+padRight+padLeft, maxSize+padDown+padUp))
# 	# remove empty space added by the warp
# 	warped = removeEmpty(warped)
	# return the warped image
	return warped, dst

# Cursor Position

In [6]:
# # Define the callback function for mouse events
# def mouse_callback(event, x, y, flags, param):
#     if event == cv2.EVENT_MOUSEMOVE:
#         # Clear the previous position by drawing a black rectangle over it
#         img_copy = img.copy()
#         cv2.rectangle(img_copy, (0, 0), (200, 50), (0, 0, 0), -1)

#         # Update the image with the current cursor position
#         font = cv2.FONT_HERSHEY_SIMPLEX
#         pos = "Position: ({}, {})".format(x, y)
#         cv2.putText(img_copy, pos, (10, 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

#         # Display the updated image
#         cv2.imshow("Image", img_copy)

# Calculate the distance between the main object and the origin

In [7]:
def position_calculator(x_object, y_object, x_origin, y_origin):
    
    x_distance = (x_object - x_origin)
    y_distance = abs(y_object - y_origin)
    
    # convert pixel to cm
    x_distance = x_distance / pixelsPerMetric
    y_distance = y_distance / pixelsPerMetric
    
    return x_distance , y_distance

# the Main while loop

In [8]:
#=====================================================
#========================INPUT========================
#=====================================================
start_point_x = 60
start_point_y = 60

Is_red_detected = False
arduino_list = []
detector_list = []
center_points = []
prev_center_x, prev_center_y, prev_center_x1, prev_center_y1 = 0, 0, 0, 0
#=====================================================

#serial communication
# import serial
# import time
# arduino = serial.Serial(port='COM3', baudrate=9600, timeout=.2)

#===============================================
# Define the lower and upper bounds of the green color in HSV color space

# green_lower = (24, 66, 0)
# green_upper = (101, 255, 255)

green_lower = (36, 68, 0)
green_upper = (167, 255, 255)

# green_lower = (37, 78, 0)
# green_upper = (50, 255, 255)

# red_lower = (174, 147, 101)
# red_upper = (179, 217, 142)

# green_lower = (38,82,45)
# green_upper = (62,255,255)

# red_lower = (68, 0, 0)
# red_upper = (152, 255, 255)

red_lower = (20,155,0)
red_upper = (179,255,255)

# red_lower = (102,168,29)
# red_upper = (122,255,131)
#==================================================

# Define the kernel for morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))

# Define the initial position and center of the object
x, y, w, h = 0, 0, 0, 0
center_x, center_y, prev_center_y, prev_center_x = 0, 0, 0, 0


# Open the video file
cap = cv2.VideoCapture(0)

# cap.set(cv2.CAP_PROP_FPS, 10)

# img = cv2.imread("test3.jpg", cv2.COLOR_BGR2GRAY)

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()

#--------------------------------------------------
#Find the marker in the first frame and caculate the PPM

# Read a frame from the video
ret, img = cap.read()
# get and process the camera calibration to warp the image
matrix, distortion = get_calibration_data()
calibrated_img = calibrate_image(img, matrix, distortion)

# find the marker in the image
marker = find_aruco_marker(calibrated_img)

calibrated_img = calibrate_image(img, matrix, distortion)

# transform the image to be bird's eye view
trans_image, dst = four_point_transform(calibrated_img, marker)

# find the location of the marker in the transformed image
trans_marker = find_aruco_marker(trans_image)

# calculate the pixels per metric for the image.  This represents how many pixels are in a cm.
distance_top = dist.euclidean(trans_marker[0], trans_marker[1]) / 20
distance_right = dist.euclidean(trans_marker[1], trans_marker[2]) / 20
distance_bottom = dist.euclidean(trans_marker[2], trans_marker[3]) / 20
distance_left = dist.euclidean(trans_marker[3], trans_marker[0]) / 20

pixelsPerMetric = (distance_top+distance_right+distance_bottom+distance_left) / 4

# pixelsPerMetric = dist.euclidean(trans_marker[2], trans_marker[1]) / 20
print("PPM: ", pixelsPerMetric)

#--------------------------------------------------
    
# Loop through the frames of the video
while True:
    
    # Read a frame from the video
    ret, img = cap.read()
    #If the frame cannot be read, break out of the loop
    if not ret:
        break

    # get and process the camera calibration to warp the image
    calibrated_img = calibrate_image(img, matrix, distortion)

    # transform the image to be bird's eye view
    trans_image, dst = four_point_transform(calibrated_img, marker)
#     cv2.imwrite("./output/transformed.jpg", trans_image)

#     # find the location of the marker in the transformed image
#     trans_marker = find_aruco_marker(trans_image)

    # Check if the user pressed the 'q' key to quit
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break
    # elif cv2.waitKey(1) & 0xFF == ord('s'):
    #     # Print the width and height of the whiteboard
    #     print("Whiteboard width:", width)
    #     print("Whiteboard height:", height)
    
    #=========================================================================================
    # Convert the frame from BGR color space to HSV color space
    hsv = cv2.cvtColor(trans_image, cv2.COLOR_BGR2HSV)

    # Threshold the image to get the green color region
    mask = cv2.inRange(hsv, green_lower, green_upper)

    # Perform morphological operations to remove noise
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Find contours in the thresholded image
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        # Get the contour with the largest area
        max_contour = max(contours, key=cv2.contourArea)
        
        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        # pass the rotated image as well as the bottom right coordinates of the aruco marker
        objectsize(trans_image, max_contour, pixelsPerMetric)
        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        
        # Get the bounding rectangle of the contour
        x, y, w, h = cv2.boundingRect(max_contour)

        # Draw a green rectangle around the object
        cv2.rectangle(trans_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Get the center of the bounding rectangle
        center_x, center_y = x + w // 2, y + h // 2
        
        # Append the center point to the list
        center_points.append((center_x, center_y))
        
    # Draw circles for each center point in the list
    #for point in center_points:
        #cv2.circle(img, point, 4, (0, 0, 255), -1)
    if (not contours):
        center_x, center_y = 0, 0
        center_points = []
    
    if (contours):

        # Draw a line connecting the centers of the bounding rectangles in consecutive frames
        if len(center_points) > 1:
            for i in range(1, len(center_points)):
                cv2.line(trans_image, center_points[i-1], center_points[i], (255, 0, 0), 2)

    # Draw a circle at the center of the bounding rectangle
    #cv2.circle(trans_image, (center_x, center_y), 4, (0, 0, 255), -1)
    
    # Draw a line connecting the centers of the bounding rectangles in consecutive frames
    #if x != 0 and y != 0:
        #cv2.line(trans_image, (center_x, center_y), (prev_center_x, prev_center_y), (255, 0, 0), 2)
        
    # Show the video capture with the green object tracked
    #cv2.imshow('Green Object Tracker', trans_image)

    # Set the previous center of the object to the current center
    prev_center_x, prev_center_y = center_x, center_y

    #=========================================================================================
    if Is_red_detected == False:
        # Convert the frame from BGR color space to HSV color space
        hsv1 = cv2.cvtColor(trans_image, cv2.COLOR_BGR2HSV)

        # Threshold the image to get the green color region
        mask1 = cv2.inRange(hsv1, red_lower, red_upper)

        # Perform morphological operations to remove noise
        mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, kernel)

        # Find contours in the thresholded image
        contours1, hierarchy1 = cv2.findContours(mask1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours1) > 0:
            # Get the contour with the largest area
            max_contour1 = max(contours1, key=cv2.contourArea)

            # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
            # pass the rotated image as well as the bottom right coordinates of the aruco marker
            objectsize(trans_image, max_contour1, pixelsPerMetric)
            # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

            # Get the bounding rectangle of the contour
            x1, y1, w1, h1 = cv2.boundingRect(max_contour1)

            # Draw a green rectangle around the object
            cv2.rectangle(trans_image, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 0), 2)

            # Get the center of the bounding rectangle
            center_x1, center_y1 = x1 + w1 // 2, y1 + h1 // 2

        # Draw a circle at the center of the bounding rectangle
        cv2.circle(trans_image, (center_x1, center_y1), 4, (0, 0, 255), -1)

        # Draw a line connecting the centers of the bounding rectangles in consecutive frames
        if x1 != 0 and y1 != 0:
            cv2.line(trans_image, (center_x1, center_y1), (prev_center_x1, prev_center_y1), (255, 0, 0), 2)

        # Show the video capture with the green object tracked
    #     cv2.imshow('Green Object Tracker', trans_image)

        # Set the previous center of the object to the current center
        prev_center_x1, prev_center_y1 = center_x1, center_y1
        
        print(pixelsPerMetric)
        # claculates a manual ppm based on the starting point of the object of intereset (OOI)
        manual_ppm = dist.euclidean((center_x1,center_y1), (center_x, center_y)) / (np.sqrt(start_point_x**2 + start_point_y**2))
        print(manual_ppm)
        # Clacluates the new PixelsPerMetric by adding the new manual ppm
        pixelsPerMetric = (distance_top+distance_right+distance_bottom+distance_left+manual_ppm) / 5
        print(pixelsPerMetric)
        
        Is_red_detected = True
    
    # Draw a green rectangle around the object
    cv2.rectangle(trans_image, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 0), 2)
    # Draw a circle at the center of the bounding rectangle
    cv2.circle(trans_image, (center_x1, center_y1), 4, (0, 0, 255), -1)
    
    #=========================================================================================
    # Create a window and set the mouse callback function
    #cv2.namedWindow("Scanned Image")
    #cv2.setMouseCallback("Scanned Image", mouse_callback)
    #=========================================================================================
    # Display the result
    cv2.namedWindow("Scanned Image", cv2.WINDOW_NORMAL)   
    cv2.imshow("Scanned Image", trans_image)
    
    x_coordinate, y_coordinate = position_calculator(center_x, center_y, center_x1, center_y1 )
    
    #================================================
    #Serial Communication
    
#     iterator = 0
#     while iterator<2:
#         line = arduino.readline()   # read a byte
#         if line:
#             string = line.decode()  # convert the byte string to a unicode string
#             string = string.strip()
#             num = (string) # convert the unicode string to an int
#             if iterator%2 == 0:
# #                 val = (num - 60.5)
# #                 print(num)
#                 arduino_list.append(num)
#             elif iterator%2 == 1:
# #                 val = (num - 15)
# #                 print(num)
#                 arduino_list.append(num)
#             iterator = iterator + 1
#     #================================================
# #     print("num of values in Arduino: ", len(arduino_list))
    
#     if  len(arduino_list) != 0:
#         detector_list.append(x_coordinate)
#         detector_list.append(y_coordinate)
#         print("detected values: ")
#         print(x_coordinate)
#         print(y_coordinate)
#         print("real values: ")
#     cv2.show("123", trans_image)
   # Press 'q' to quit the program
    # if True:
    #     print("x: " + "{:.1f}cm".format(x_coordinate) + " y: {:.1f}cm".format(y_coordinate))
#         print("Object 0 center coordinates: ({}, {})".format(center_x, center_y))
#         print("Object 1 center coordinates: ({}, {})".format(center_x1, center_y1))
    if cv2.waitKey(1) == ord('p'):
        print("detected values: ")
        print(x_coordinate, y_coordinate)
        detector_list.append(x_coordinate)
        detector_list.append(y_coordinate)
        
    elif cv2.waitKey(1) == ord('q'):
        break

    # Release the video capture object and close all windows
cap.release()   
cv2.destroyAllWindows()

PPM:  10.400120162963868
10.400120162963868
10.408603545998751
10.401816839570845
detected values: 
59.60497185851042 60.37406826958797
detected values: 
39.41619106772463 60.37406826958797
detected values: 
19.419684379708233 60.08565711543389
detected values: 
-0.2884111541540827 59.989520064049195
detected values: 
59.60497185851042 40.28142453018688
detected values: 
39.32005401633994 40.18528747880219
detected values: 
19.419684379708233 40.08915042741749
detected values: 
-0.09613705138469422 40.08915042741749
detected values: 
59.60497185851042 19.9965066880164
detected values: 
39.41619106772463 20.092643739401094
detected values: 
19.611958482477622 20.18878079078579
detected values: 
0.0 19.9965066880164
detected values: 
59.797245961279806 0.19227410276938844
detected values: 
39.704602221878716 0.0
detected values: 
19.708095533862316 0.0
detected values: 
19.80423258524701 0.0


In [13]:
f = open("milimetri7(bigger scale).txt", "w")

for i in range(len(detector_list)):
    f.write(str(detector_list[i]) + ',')

f.close()

In [18]:
x = 0
y = 0
ideal = []

for i in range(3,-1,-1):
    for j in range(3,-1,-1):
        ideal.append(x + (20*j))
        ideal.append(y + (20*i))
        print(x + (20*j), y + (20*i))

60 60
40 60
20 60
0 60
60 40
40 40
20 40
0 40
60 20
40 20
20 20
0 20
60 0
40 0
20 0
0 0


In [19]:
f = open("idealmilimetri7(bigger scale).txt", "w")

for i in range(len(ideal)):
    f.write(str(ideal[i]) + ',')

f.close()

In [20]:
print(trans_marker[0], trans_marker[1], trans_marker[2], trans_marker[3])
print(dist.euclidean(trans_marker[0], trans_marker[1]) / 20)
print(dist.euclidean(trans_marker[1], trans_marker[2]) / 20)
print(dist.euclidean(trans_marker[2], trans_marker[3]) / 20)
print(dist.euclidean(trans_marker[3], trans_marker[0]) / 20)

[1280.  960.] [1487.  959.] [1488. 1168.] [1281. 1169.]
10.350120544433594
10.450119781494141
10.350120544433594
10.450119781494141
